###1. взять любой набор данных для бинарной классификации (можно скачать один с https://archive.ics.uci.edu/ml/datasets.php)
###2. сделать feature engineering
###3. обучить любой классификатор (какой вам нравится)
###4. далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть
###5. применить random negative sampling для построения классификатора в новых условиях
###6. сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)
###7. поэкспериментировать с долей P на шаге 6 (как будет меняться качество модели при уменьшении/увеличении размера P)

Расмотрим пример на датасете из репозитория UCI

Описание данных -https://archive.ics.uci.edu/ml/datasets/Credit+Approval

###Соответствующая информация:

    Этот файл касается приложений для кредитных карт. Все имена атрибутов
    и значения были изменены на бессмысленные символы для защиты
    конфиденциальность данных.
  
    Этот набор данных интересен тем, что в нем хорошо сочетаются
    атрибуты -- непрерывные, номинальные с небольшим количеством
    значений, и номинальные с большим количеством значений. 
  
5. Количество экземпляров: 690

6. Количество атрибутов: 15 + атрибут класса

7. Информация об атрибутах:

    A1:	b, a.
    A2:	continuous.
    A3:	continuous.
    A4:	u, y, l, t.
    A5:	g, p, gg.
    A6:	c, d, cc, i, j, k, m, r, q, w, x, e, aa, ff.
    A7:	v, h, bb, j, n, z, dd, ff, o.
    A8:	continuous.
    A9:	t, f.
    A10:	t, f.
    A11:	continuous.
    A12:	t, f.
    A13:	g, p, s.
    A14:	continuous.
    A15:	continuous.
    A16: +,-         (class attribute)


8. Распределение классов
  
    +: 307 (44,5%)
    -: 383 (55,5%)

In [1]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
data = pd.read_csv("crx.data", header=None)
data.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.00,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.46,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.50,u,g,q,h,1.50,t,f,0,f,g,00280,824,+


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       690 non-null    object 
 1   1       690 non-null    object 
 2   2       690 non-null    float64
 3   3       690 non-null    object 
 4   4       690 non-null    object 
 5   5       690 non-null    object 
 6   6       690 non-null    object 
 7   7       690 non-null    float64
 8   8       690 non-null    object 
 9   9       690 non-null    object 
 10  10      690 non-null    int64  
 11  11      690 non-null    object 
 12  12      690 non-null    object 
 13  13      690 non-null    object 
 14  14      690 non-null    int64  
 15  15      690 non-null    object 
dtypes: float64(2), int64(2), object(12)
memory usage: 86.4+ KB


У нас есть 15 признаков и 1 целевая переменная (бинарная) 

In [3]:
print(data.shape)

(690, 16)


Посмотрим на соотношение классов

In [4]:
data.iloc[:,-1].value_counts()

-    383
+    307
Name: 15, dtype: int64

In [5]:
data[15] = data[15].apply(lambda x: 0 if x == '-' else 1)

In [6]:
cat_col = [0, 1, 3, 4, 5, 6, 8, 9, 11, 12, 13]
feature_col = [2, 7, 10, 14]

In [7]:
one_hot = pd.get_dummies(data[cat_col])
data = data.drop(cat_col, axis=1)
data = data.join(one_hot)

In [8]:
scaler = StandardScaler()

data_norm = data.copy()
data[feature_col] = scaler.fit_transform(data_norm[feature_col])

In [9]:
col_ = data.pop(15)
data.insert(len(data.columns), 15, col_)

In [10]:
data.head(3)

,2,7,10,14,0_?,0_a,0_b,1_13.75,1_15.17,1_15.75,...,13_00711,13_00720,13_00760,13_00840,13_00928,13_00980,13_01160,13_02000,13_?,15
0,-0.956613,-0.291083,-0.288101,-0.195413,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,-0.060051,0.244190,0.740830,-0.087852,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,-0.856102,-0.216324,-0.493887,-0.037144,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


Разбиваем выборку на тренировочную и тестовую части и обучаем модель 

In [11]:
from sklearn.model_selection import train_test_split

x_data = data.iloc[:,:-1]
y_data = data.iloc[:,-1]

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.25, random_state=4)

In [12]:
import xgboost as xgb

model = xgb.XGBClassifier()

model.fit(x_train, y_train)
y_predict = model.predict(x_test)

Проверяем качество

In [13]:
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score
score = []

def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    print("f1: %.2f%%" % (f1 * 100.0)) 
    roc = roc_auc_score(y_test, y_predict)
    print("roc: %.2f%%" % (roc * 100.0)) 
    rec = recall_score(y_test, y_predict, average='binary')
    print("recall: %.2f%%" % (rec * 100.0)) 
    prc = precision_score(y_test, y_predict, average='binary')
    print("precision: %.2f%%" % (prc * 100.0))
    score.append([f1, prc, rec, roc]) 

    
evaluate_results(y_test, y_predict)

Classification results:
f1: 82.12%
roc: 84.47%
recall: 84.93%
precision: 79.49%


### Теперь очередь за PU learning

Представим, что нам неизвестны негативы и часть позитивов

In [14]:
mod_data = data.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 25% of the positives marked
pos_sample_len = int(np.ceil(0.25 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 77/307 as positives and unlabeling the rest


Создаем столбец для новой целевой переменной, где у нас два класса - P (1) и U (-1)

In [15]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    613
 1     77
Name: class_test, dtype: int64


In [16]:
x_data = mod_data.iloc[:,:-2].values # just the X 
y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_data.iloc[:,-2].values # original class

###random negative sampling

In [17]:
mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(77, 572) (77, 572)


In [18]:
model = xgb.XGBClassifier()

model.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict = model.predict(sample_test.iloc[:,:-2].values)
evaluate_results(sample_test.iloc[:,-2].values, y_predict)

Classification results:
f1: 77.46%
roc: 82.93%
recall: 95.94%
precision: 64.95%


In [19]:
df = pd.DataFrame({'original': score[0], 'random negative sampling': score[1]}).T
df.columns = ['fscore', 'precision', 'recall', 'roc_auc']
df

,fscore,precision,recall,roc_auc
original,0.821192,0.794872,0.849315,0.844658
random negative sampling,0.774590,0.649485,0.959391,0.829253


In [21]:
mod_data = data.copy()

pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]

np.random.shuffle(pos_ind)
# leave just 30% of the positives marked
pos_sample_len = int(np.ceil(0.3 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 93/307 as positives and unlabeling the rest


In [22]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    597
 1     93
Name: class_test, dtype: int64


In [ ]:
x_data = mod_data.iloc[:,:-2].values # just the X 
y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_data.iloc[:,-2].values # original class

In [23]:
mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(93, 572) (93, 572)


In [24]:
model = xgb.XGBClassifier()

model.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict = model.predict(sample_test.iloc[:,:-2].values)
evaluate_results(sample_test.iloc[:,-2].values, y_predict)

Classification results:
f1: 79.26%
roc: 84.83%
recall: 94.51%
precision: 68.25%


In [25]:
df = pd.DataFrame({'original': score[0], 'random negative sampling 0.25': score[1], 'random negative sampling 0.30': score[2]}).T
df.columns = ['fscore', 'precision', 'recall', 'roc_auc']
df

,fscore,precision,recall,roc_auc
original,0.821192,0.794872,0.849315,0.844658
random negative sampling 0.25,0.774590,0.649485,0.959391,0.829253
random negative sampling 0.30,0.792627,0.682540,0.945055,0.848304


### Вывод: наилучший F-score и Precision у обычного XGboost, однако Recall выше при использовании random negative sampling с долей 0.25